## 개체명 인식

* 도메인 또는 목적에 맞게 개체명을 인식시킴  

* 개체명의 종류  
  - 분석 목적에 맞춰서 데이터 셋 자체 구축 + 개체명 인식 모델 훈련  
  - 조직, 사람, 지역과 관련된 개체명 인식 => 기존에 훈련된 모델을 갖고 분석 가능  

* BIO 표현  
  - B 시작 / I 내부 / O 기타, 그외  

* 훈련 데이터셋을 만드는 방법
  - 구문 분석을 활용 (특정 부분에 등장하는 구문에서 데이터 추출: 라벨링)  
  - 기존에 훈련된 모델의 결과를 새로운 데이터에 적용하여 라벨링을 달고, 그 데이터를 다시 훈련 데이터에 포함하여 모델 훈련 방식  


In [1]:
import pandas as pd

In [4]:
toks = "Jeff Dean is a computer scientist at Google in California".split()
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "O", "B-LOC"]
df = pd.DataFrame(data=[toks, lbls], index=['Tokens', 'Tags'])
df.T

,Tokens,Tags
0,Jeff,B-PER
1,Dean,I-PER
2,is,O
3,a,O
4,computer,O
5,scientist,O
6,at,O
7,Google,B-ORG
8,in,O
9,California,B-LOC


In [5]:
import re 
import urllib.request

데이터셋 준비

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/12.%20RNN%20Sequence%20Labeling/dataset/train.txt", filename="train.txt" )

f = open('train.txt', 'r')
tagged_sentences = []
sentence = []

for line in f :
    if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
        if len(sentence) > 0 :
            tagged_sentences.append(sentence)
            sentence = []
        continue
    splits = line.split(' ')
    splits[-1] = re.sub(r'\n', '', splits[-1])
    word = splits[0].lower()
    sentence.append([word, splits[-1]])

In [7]:
tagged_sentences[:5]

[[['eu', 'B-ORG'],
  ['rejects', 'O'],
  ['german', 'B-MISC'],
  ['call', 'O'],
  ['to', 'O'],
  ['boycott', 'O'],
  ['british', 'B-MISC'],
  ['lamb', 'O'],
  ['.', 'O']],
 [['peter', 'B-PER'], ['blackburn', 'I-PER']],
 [['brussels', 'B-LOC'], ['1996-08-22', 'O']],
 [['the', 'O'],
  ['european', 'B-ORG'],
  ['commission', 'I-ORG'],
  ['said', 'O'],
  ['on', 'O'],
  ['thursday', 'O'],
  ['it', 'O'],
  ['disagreed', 'O'],
  ['with', 'O'],
  ['german', 'B-MISC'],
  ['advice', 'O'],
  ['to', 'O'],
  ['consumers', 'O'],
  ['to', 'O'],
  ['shun', 'O'],
  ['british', 'B-MISC'],
  ['lamb', 'O'],
  ['until', 'O'],
  ['scientists', 'O'],
  ['determine', 'O'],
  ['whether', 'O'],
  ['mad', 'O'],
  ['cow', 'O'],
  ['disease', 'O'],
  ['can', 'O'],
  ['be', 'O'],
  ['transmitted', 'O'],
  ['to', 'O'],
  ['sheep', 'O'],
  ['.', 'O']],
 [['germany', 'B-LOC'],
  ["'s", 'O'],
  ['representative', 'O'],
  ['to', 'O'],
  ['the', 'O'],
  ['european', 'B-ORG'],
  ['union', 'I-ORG'],
  ["'s", 'O'],
  ['vete

In [8]:
sentence[:5]

[]

In [9]:
print("전체 샘플 개수 : ", len(tagged_sentences))

전체 샘플 개수 :  14041


In [10]:
tagged_sentences[0]

[['eu', 'B-ORG'],
 ['rejects', 'O'],
 ['german', 'B-MISC'],
 ['call', 'O'],
 ['to', 'O'],
 ['boycott', 'O'],
 ['british', 'B-MISC'],
 ['lamb', 'O'],
 ['.', 'O']]

In [12]:
sentence, tag_info = zip(*tagged_sentences[0])
sentence, tag_info

(('eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'),
 ('B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'))

In [13]:
sentences, ner_tags = [], []
for tagged_sentence in tagged_sentences:
    sentence, tag_info = zip(*tagged_sentence)
    sentences.append(list(sentence))
    ner_tags.append(list(tag_info))

개체명 인식 모델 만들기

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [15]:
vocab_size = 4000
src_tokenizer = Tokenizer(num_words=4000, oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

In [16]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(ner_tags)

In [18]:
tag_size = len(tar_tokenizer.word_index) + 1 
print('단어 집합의 크기 : ' ,  vocab_size)
print('개체명 태깅 정보의 집합의 크기 : ', tag_size)

단어 집합의 크기 :  4000
개체명 태깅 정보의 집합의 크기 :  10


In [19]:
# 정수인코딩 : {"사과" : 1 , }
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(ner_tags)

In [21]:
print(sentences[0])
print(X_train[0])
print(ner_tags[0])
print(y_train[0])

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
[989, 1, 205, 629, 7, 3939, 216, 1, 3]
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
[4, 1, 7, 1, 1, 1, 7, 1, 1]


In [22]:
index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word

In [23]:
decoded = []
for index in X_train[0]:
    decoded.append(index_to_word[index])

print(f"기존 문장 : {sentences[0]}")
print(f"빈도수가 낮은 단어, OOV 처리된 부분 확인 : {decoded}")

기존 문장 : ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
빈도수가 낮은 단어, OOV 처리된 부분 확인 : ['eu', 'OOV', 'german', 'call', 'to', 'boycott', 'british', 'OOV', '.']


In [24]:
# 텍스트 데이터의 길이를 통일 하기 위해  padding을 적용

max_len = 70 # 세밀한 분석을 위해서는 텍스트 데이터 EDA를 통해서 max_len / median_len / mean_len 을 보고서 파라미터의 값을 설정함

X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = pad_sequences(y_train, padding='post', maxlen=max_len)

In [25]:
X_train, X_test , y_train , y_test = train_test_split(X_train, y_train, test_size=.3, random_state=42)


In [26]:
# 원핫인코딩

y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [28]:
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))


훈련 샘플 문장의 크기 : (9828, 70)
훈련 샘플 레이블의 크기 : (9828, 70, 10)
테스트 샘플 문장의 크기 : (4213, 70)
테스트 샘플 레이블의 크기 : (4213, 70, 10)


### 모델 훈련

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

In [30]:
from tensorflow import random as tf_random , constant_initializer
import random
import numpy as np

def reset_model_random():
    random_seed_num = 0
    tf_random.set_seed(random_seed_num)
    np.random.seed(random_seed_num)
    random.seed(random_seed_num)
    constant_initializer()


In [31]:
embedding_dim = 128
hidden_units = 128

In [32]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size , output_dim = embedding_dim, 
                    input_length = max_len , mask_zero = True))
model.add(Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation = 'softmax')))


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 128)           512000    
                                                                 
 bidirectional (Bidirection  (None, 70, 256)           263168    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 70, 10)            2570      
 ributed)                                                        
                                                                 
Total params: 777738 (2.97 MB)
Trainable params: 777738 (2.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
reset_model_random()

es = EarlyStopping(patience=2, monitor='accuracy')
mc = ModelCheckpoint(f"./model/NER_model.h5", save_best_only=True)

model.compile(loss = 'categorical_crossentropy', optimizer=Adam(0.001)
              , metrics = ['accuracy'])

history = model.fit(X_train, y_train, batch_size=128 , epochs= 10 ,
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/10
77/77 [==============================] - 33s 376ms/step - loss: 1.0105 - accuracy: 0.8200 - val_loss: 0.6744 - val_accuracy: 0.8359
Epoch 2/10


c:\ProgramData\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 30s 392ms/step - loss: 0.5529 - accuracy: 0.8385 - val_loss: 0.4450 - val_accuracy: 0.8633
Epoch 3/10
77/77 [==============================] - 30s 390ms/step - loss: 0.3914 - accuracy: 0.8805 - val_loss: 0.3365 - val_accuracy: 0.9027
Epoch 4/10
77/77 [==============================] - 31s 398ms/step - loss: 0.2829 - accuracy: 0.9175 - val_loss: 0.2521 - val_accuracy: 0.9289
Epoch 5/10
77/77 [==============================] - 31s 399ms/step - loss: 0.2084 - accuracy: 0.9387 - val_loss: 0.1982 - val_accuracy: 0.9441
Epoch 6/10
77/77 [==============================] - 30s 390ms/step - loss: 0.1630 - accuracy: 0.9521 - val_loss: 0.1773 - val_accuracy: 0.9495
Epoch 7/10
77/77 [==============================] - 30s 395ms/step - loss: 0.1343 - accuracy: 0.9604 - val_loss: 0.1668 - val_accuracy: 0.9531
Epoch 8/10
77/77 [==============================] - 31s 397ms/step - loss: 0.1161 - accuracy: 0.9662 - val_loss: 0.1623 - val_accuracy: 0.9531
Epoch 9/10

모델 훈련 히스토리 관리 방법
  
- log 파일
- 모델을 좀 더 관리하는 경우에는 DB에 모델 정보를 관리하는 테이블에 저장
  
모델 서빙 방법/시점에서 활용 
1. 특정 일자마다 모델 재훈련을 한다면, 모델 훈련 (평균) 소요시간 이후에 모델의 결과를 DB에서 확인  
2. 모델 결과/평가 기준이 확인  
   if 평가 기준 = 미달 :  
      모델 재훈련을 하는 이메일/알람 => 운영자 혹은 담당자가 재훈련(모델 아키텍처부터 재설계 후 모델 재훈련)  
   else :    
      저장된 모델을 운영 서버에 적용하도록 함   


In [35]:
# 모델 훈련 히스토리 : log 파일로 남기거나 혹은 모델을 좀 더 관리하는 경우에는 DB에 모델 정보를 관리하는 테이블에 저장

history.history

{'loss': [1.0104928016662598,
  0.5528500080108643,
  0.39142704010009766,
  0.2829253375530243,
  0.20840083062648773,
  0.16298261284828186,
  0.13434351980686188,
  0.11606518179178238,
  0.10362490266561508,
  0.09163041412830353],
 'accuracy': [0.8199923038482666,
  0.8385356068611145,
  0.8805081844329834,
  0.9174547791481018,
  0.938693106174469,
  0.9520843029022217,
  0.9604144096374512,
  0.9661895036697388,
  0.9699485301971436,
  0.9733155965805054],
 'val_loss': [0.6744264960289001,
  0.44497713446617126,
  0.3364686667919159,
  0.2520807385444641,
  0.1982152760028839,
  0.17729799449443817,
  0.166817307472229,
  0.1623183786869049,
  0.16096438467502594,
  0.17201517522335052],
 'val_accuracy': [0.8359054327011108,
  0.8633451461791992,
  0.9026764631271362,
  0.9288808107376099,
  0.9440500736236572,
  0.9495182633399963,
  0.9531252384185791,
  0.9530923366546631,
  0.9557605385780334,
  0.955348789691925]}

In [36]:
# 모델 평가
model.evaluate(X_test, y_test)

132/132 [==============================] - 3s 19ms/step - loss: 0.1717 - accuracy: 0.9553


[0.1717061996459961, 0.955348789691925]

In [38]:
# 예측하기
y_predicted = model.predict(np.array([X_test[0]]))

# 확률 벡터를 정수 레이블로 변경
y_predicted = np.argmax(y_predicted, axis= -1)

# 원-핫 벡터를 정수 인코딩으로 변경
labels = np.argmax(y_test[0], -1)

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "=")

for word, tag, pred in zip(X_test[0], labels, y_predicted[0]):
    if word != 0 : 
        print("{:17}:{:7} {}".format(index_to_word[word], index_to_ner[tag], index_to_ner[pred]))

1/1 [==============================] - 0s 15ms/step
단어             |실제값  |예측값
attendance       :o       o
:                :o       o
OOV              :o       o


In [40]:
# 예측하기
y_predicted = model.predict(np.array([X_test[90]]))

# 확률 벡터를 정수 레이블로 변경
y_predicted = np.argmax(y_predicted, axis= -1)

# 원-핫 벡터를 정수 인코딩으로 변경
labels = np.argmax(y_test[90], -1)

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "=")

for word, tag, pred in zip(X_test[90], labels, y_predicted[0]):
    if word != 0 : 
        print("{:17}:{:7} {}".format(index_to_word[word], index_to_ner[tag], index_to_ner[pred]))

1/1 [==============================] - 0s 15ms/step
단어             |실제값  |예측값
"                :o       o
i                :o       o
looked           :o       o
at               :o       o
it               :o       o
as               :o       o
not              :o       o
a                :o       o
first            :o       o
round            :o       o
match            :o       o
,                :o       o
just             :o       o
a                :o       o
great            :o       o
challenge        :o       o
for              :o       o
me               :o       o
,                :o       o
"                :o       o
said             :o       o
coetzer          :b-per   b-per
,                :o       o
24               :o       o
.                :o       o
"                :o       o


이후에 할 수 있는 분석 작업들

1. 새로운 데이터가 들어오면, 개체명을 예측해서 그 값을 추출하여 통계적 분석(Count)
   - word 자체에 대한 카운팅 (예: 'coetzer')
   - 개체명 결과(라벨)에 대한 카운팅 (예 : 'b-per/I-per' / 'b-org/I-per' )